# Automated download of data from NASA APPEEARS for USGS site/basin of interest and reformatting for use within ILAMB-Watershed

In [2]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

In [23]:
AOIFILE='/home/jbk/projects/climate/interface/src/hydrology_codes/basin_07340300.json'

# NASA APPEEARS Products list
https://appeears.earthdatacloud.nasa.gov/products

# NASA APPEEARS REST API tutorial
https://lpdaac.usgs.gov/resources/e-learning/getting-started-with-the-a%CF%81%CF%81eears-api-submitting-and-downloading-an-area-request/

In [65]:
# Set input directory, change working directory
inDir = os.getcwd()                                    # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                        # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

In [4]:
# Login with NASA Earthdata Account
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username:  ········
Enter NASA Earthdata Login Password:  ········


In [5]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response  

{'token_type': 'Bearer',
 'token': 'mV31XFKShbOBqS1yQmldQDOMQrfM7Uz0TGATDto22muGrZQjoI_4tzv9kBS6EIsWx_T-GZJ2dHOlH9oJgeXMfQ',
 'expiration': '2023-04-03T17:22:37Z'}

In [6]:
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AρρEEARS currently supports {} products.'.format(len(product_response)))  # Print no. products available in AppEEARS

AρρEEARS currently supports 168 products.


In [11]:
all_products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version

In [9]:

products = ['MYD16A2.061', 'MCD15A2H.061', 'MYD17A2.061']                                         # Print information for MYD16A2.061 ET Product

In [18]:
# Create list of products of interest
# MYD16A2.061  ET -- MODIS Aqua
# MCD15A2H.061 LAI -- combined MODIS
# MYD17A2.061  GPP -- MODIS Aqua

prod_list = ['MYD16A2.061', 'MCD15A2H.061', 'MYD17A2H.061']
# Get the layers available from MYD16A2.061
print(r.get('{}product/{}'.format(api, prod_list[0])).json().keys())
# Get the layers available from MCD15A2H.61
print(r.get('{}product/{}'.format(api, prod_list[1])).json().keys())
# Get the layers available from MYD17A2.061
print(r.get('{}product/{}'.format(api, prod_list[2])).json().keys())



['ET_500m', 'ET_QC_500m', 'LE_500m', 'PET_500m', 'PLE_500m']


In [22]:

# Get list of layers for this product
# Add ET vars to the list of layers we want to request
layers = [(prod_list[0],'ET_500m'),(prod_list[0],'PET_500m')]  # Create tupled list linking desired product with desired layers
# Add LAI vars to the list of layers we want to request
layers.append((prod_list[1],'LAI_500m'))
# Add GPP vars to the list of layers we want to request
layers.append((prod_list[2],'GPP_500m'))

# Create a list of dictionaries 
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'ET_500m', 'product': 'MYD16A2.061'},
 {'layer': 'PET_500m', 'product': 'MYD16A2.061'},
 {'layer': 'LAI_500m', 'product': 'MCD15A2H.061'},
 {'layer': 'GPP_500m', 'product': 'MYD17A2H.061'}]

In [33]:
# Submit an Area Request
# The Submit task API call provides a way to submit a new request to be processed. It can accept data via JSON, query string, or a combination of both. In the example below, compile a json and submit a request. Tasks in AρρEEARS correspond to each request associated with your user account. Therefore, each of the calls to this service requires an authentication token (see Section 1c.), which is stored in a header below.
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

In [49]:
# Read a GeoJSON for the AOI
aoi = gpd.read_file(AOIFILE) # Read in GeoJSON or Shapefile as dataframe using geopandas
print(aoi.head())                                                # Print first few lines of dataframe
aoi = json.loads(aoi.to_json())


  identifier                                           geometry
0   07340300  POLYGON ((-94.25121 34.34012, -94.25177 34.341...


In [28]:
# Projections supported by APPEEARS
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
#projections 
# Create a dictionary of projections with projection Name as the keys.
projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north']

In [68]:
# Compile a JSON
task_name = 'modis_basin_07340300' # User-defined name of the task: 'NPS Vegetation Area' used in example
task_type = ['point','area']        # Type of task, area or point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['netcdf4']  # Set output file format type
startDate = '01-01-2018'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '12-31-2018'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

# create JSON
task = {
    'task_type': task_type[1],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': aoi,
    }
}

In [69]:
# Submit a Task Request
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response  

{'task_id': 'f062692a-0a46-4c10-b51e-4abf33dd77fc', 'status': 'pending'}

In [70]:
# Retrieve Task Status
params = {'limit': 2, 'pretty': True} # Limit API response to 2 most recent entries, return as pretty json
tasks_response = r.get('{}task'.format(api), params=params, headers=head).json() # Query task service, setting params and header 
tasks_response     



[{'tier': 2,
  'error': None,
  'params': {'dates': [{'endDate': '12-31-2018', 'startDate': '01-01-2018'}],
   'layers': [{'layer': 'ET_500m', 'product': 'MYD16A2.061'},
    {'layer': 'PET_500m', 'product': 'MYD16A2.061'},
    {'layer': 'LAI_500m', 'product': 'MCD15A2H.061'},
    {'layer': 'GPP_500m', 'product': 'MYD17A2H.061'}],
   'output': {'format': {'type': 'netcdf4'}, 'projection': 'geographic'}},
  'status': 'queued',
  'created': '2023-04-02T14:09:55.500332',
  'task_id': 'f062692a-0a46-4c10-b51e-4abf33dd77fc',
  'updated': '2023-04-02T14:09:55.587684',
  'user_id': 'jitendra.ornl@gmail.com',
  'estimate': {'request_size': 1170116.863820423, 'request_memory': 1},
  'retry_at': None,
  'has_swath': False,
  'task_name': 'modis_basin_07340300',
  'task_type': 'area',
  'api_version': 'v1',
  'svc_version': '3.26',
  'web_version': None,
  'has_nsidc_daac': False,
  'expires_on': '2023-05-02T14:09:55.587684'},
 {'tier': 2,
  'error': None,
  'params': {'dates': [{'endDate': '01-31

In [63]:
# take the task id returned from the task_response that was generated when submitting your request, and use the AρρEEARS API status service to check the status of your request.
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
status_response          

{'tier': 2,
 'error': None,
 'params': {'geo': {'type': 'FeatureCollection',
   'features': [{'id': '0',
     'type': 'Feature',
     'geometry': {'type': 'Polygon',
      'coordinates': [[[-94.25120746438971, 34.34012232268112],
        [-94.2517712566365, 34.34199470939717],
        [-94.24818658460966, 34.3483669878912],
        [-94.25057553647237, 34.35038422822961],
        [-94.25118799228248, 34.35272067659477],
        [-94.24945702410888, 34.35488561581433],
        [-94.25162732590202, 34.355333364270344],
        [-94.2507414710471, 34.358970240816475],
        [-94.25284218124028, 34.360802767091684],
        [-94.2508437620839, 34.36400150261035],
        [-94.2525347716894, 34.374790702790456],
        [-94.25585494011908, 34.37462244692883],
        [-94.26934211461891, 34.38276122464855],
        [-94.26902989423795, 34.38535418407524],
        [-94.27155363514419, 34.38577953074313],
        [-94.27103129087473, 34.38800457158036],
        [-94.27334050660272, 34.3888

In [71]:
# call the task service for your request every 20 seconds to check the status of your request.
# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

done


In [72]:
# Download a Request
destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

# Explore Files in Request Output 
# The bundle API provides information about completed tasks. For any completed task, a bundle can be queried to return the files contained as a part of the task request. Below, call the bundle API and return all of the output files.
bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
bundle

# Download Files in a Request (Automation)
# use the contents of the bundle to select the file name and id and store as a dictionary.
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files 

# Use the files dictionary and a for loop to automate downloading all of the output files into the output directory.
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
    if files[f].endswith('.tif'):
        filename = files[f].split('/')[1]
    else:
        filename = files[f] 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))


Downloaded files can be found at: /home/jbk/projects/climate/interface/src/hydrology_codes/modis_basin_07340300


In [73]:
bundle

{'files': [{'sha256': '2b1048abce3a6eed0ba4b8f63a1d8bd38d43e4e03598d921ab10e7fbae394a83',
   'file_id': 'd2223a2c-d057-4328-ac4d-0117203c53b0',
   'file_name': 'modis-basin-07340300-request.json',
   'file_size': 6774,
   'file_type': 'json',
   's3_url': 's3://appeears-output/56d7d841-a043-4ce9-9263-f691e7c61d7f/modis-basin-07340300-request.json'},
  {'sha256': 'ac81e9135a2465e532192121ce9722a3b29cffe7955ff5faf11b7a2585b72290',
   'file_id': '4ebe17b6-9c4d-449e-8323-016d90fa05f7',
   'file_name': 'modis-basin-07340300-MCD15A2H-061-metadata.xml',
   'file_size': 21090,
   'file_type': 'xml',
   's3_url': 's3://appeears-output/56d7d841-a043-4ce9-9263-f691e7c61d7f/modis-basin-07340300-MCD15A2H-061-metadata.xml'},
  {'sha256': 'ebe0dad86f6aaedd2ec48bbc3f85306eac83224959bbbbc6fae47b5c5754de49',
   'file_id': 'daf2774e-58ba-4a99-825f-9dc63bd511f6',
   'file_name': 'modis-basin-07340300-MYD16A2-061-metadata.xml',
   'file_size': 21022,
   'file_type': 'xml',
   's3_url': 's3://appeears-outpu

In [76]:
aoi_1 = gpd.read_file(AOIFILE) # Read in GeoJSON or Shapefile as dataframe using geopandas


In [77]:
aoi_1.area

/tmp/ipykernel_569688/2240628493.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  aoi_1.area


0    0.024761
dtype: float64

In [94]:
aoi_1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [92]:
aoi_1.to_crs(3395).area/10e6

0    37.028946
dtype: float64